In [2]:
import pandas as pd
import json
from gpt3_utils import *

ValueError: attempted relative import beyond top-level package

In [ ]:
import numpy as np
import lime
import torch
import torch.nn.functional as F
from lime.lime_text import *
import scipy
import itertools
import ipdb

In [2]:
pd.set_option('max_colwidth',500)

In [3]:
#Define which dev set output to load to run GPT-3 on
#Run test model first

run_dir = '../outputs/BC5CDR-disease/gpt3/0/cv/0/'
dev_ddi = pd.read_csv(run_dir+'gpt3.output.csv',sep='\t',index_col=0)

In [5]:
params = json.load(open(run_dir+'params.json','r'))

In [7]:
dev_ddi['empty'] =  [n == 0 for n in dev_ddi.num_entities]
dev_ddi['num_predictions'] = [np.sum([1 if p != '' else 0 for p in eval(preds)]) for preds in dev_ddi.predictions]

In [8]:
in_context_rel_dist = []
null_samples_in_prompts = []

for i,row in dev_ddi.iterrows():
    final_input = row.final_input_prompts
    
    prompt_rels = []
    null_samples_in_prompt = 0
    
    lines = final_input.split('\n')
    
    for line in lines[:-1]:
        if 'Diseases:' in line:

            rel = line.split('Diseases:')[-1]
            if rel.strip() != '':
                prompt_rels.append('['+rel.strip()+']')
            else:
                prompt_rels.append('\n')
                null_samples_in_prompt += 1

    in_context_rel_dist.append(prompt_rels)
    null_samples_in_prompts.append(null_samples_in_prompt)
    
dev_ddi['in_context_rel_dist'] = in_context_rel_dist
dev_ddi['num_null_samples'] = null_samples_in_prompts

In [9]:
prompt_lists = []

for i,row in dev_ddi.iterrows():
    final_input = row.final_input_prompts
    
    prompt_rels = []
    null_samples_in_prompt = 0
    
    lines = final_input.split('\n\n')
    
    prompt_lists.append(lines[:-1])

dev_ddi['prompt_lists'] = prompt_lists

In [12]:
testing = dev_ddi.loc[[0]]

In [15]:
prompt_to_choose = [5,1,2,3,4] #List of indices to use as prompt

new_test_ready_prompts = []

for i, row in testing.iterrows():
    
    samples = [dev_ddi.loc[ind].prompts for ind in prompt_to_choose]
                
    new_test_prompt = samples
    new_test_prompt.append(row.empty_prompts)
    new_test_prompt = '\n\n'.join(new_test_prompt)
    
    new_test_ready_prompts.append(new_test_prompt)

print(testing.test_ready_prompts.values[0])
print('='*80)
print(new_test_ready_prompts[0])

testing['test_ready_prompts'] = new_test_ready_prompts

In [23]:
new_results = run_gpt3_ner_df(testing, params)

1it [00:02,  2.18s/it]
